In [1]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import io
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
from collections import Counter

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [25]:
data = files.upload()

Saving Animal_Data.csv to Animal_Data (1).csv


In [26]:
animalData_df = pd.read_csv(io.BytesIO(data['Animal_Data.csv']))
animalData_df.head()

,Animal_ID,Name,State,Age,Sex,Animal_type,Breed_Class,Color,Weight_lbs,Temperature,...,Vomiting,Diarrhea,Inappetence,Lethargic,Lameness,Muscle_pain,Joint_swelling,Reported_weight_loss,Skin_condition,4Dx_tested
0,A803419,NaN,TX,2 years,Intact Male,Dog,Small,Black/White,7,101,...,None,None,None,None,None,None,None,Present,Normal,Not Tested
1,A800463,Delilah,TX,7 months,Intact Female,Dog,Medium,Tan/Black,45,102,...,Mild,Mild,Moderate,None,None,Mild,None,None,Normal,Not Tested
2,A803414,Martie,TX,2 years,Intact Female,Dog,Large,Black/Tricolor,63,102,...,Mild,None,None,Mild,None,None,None,None,Normal,Not Tested
3,A803417,NaN,TX,1 month,Intact Female,Dog,Medium,Brown/Tan,39,101,...,Mild,Mild,Mild,Moderate,None,None,None,None,Normal,Not Tested
4,A803412,NaN,TX,5 months,Intact Male,Dog,Small,Brown/Black,21,102,...,None,None,None,Mild,None,None,None,None,Normal,Not Tested


In [57]:
animalData_df.columns

Index(['Animal_ID', 'Name', 'State', 'Age', 'Sex', 'Animal_type',
       'Breed_Class', 'Color', 'Weight_lbs', 'Temperature', 'Heart_Rate_bpm',
       'Resp_Rate_bpm', 'MM', 'CRT', 'Mentation', 'Vomiting', 'Diarrhea',
       'Inappetence', 'Lethargic', 'Lameness', 'Muscle_pain', 'Joint_swelling',
       'Reported_weight_loss', 'Skin_condition', '4Dx_tested'],
      dtype='object')

In [58]:
cleaned_df1 = animalData_df.drop(['Animal_ID', 'Name', 'State', 'Sex', 'Animal_type', 'Breed_Class', 'Color'], axis=1)
print(cleaned_df1.shape[0])
cleaned_df1.head()

49509


,Age,Weight_lbs,Temperature,Heart_Rate_bpm,Resp_Rate_bpm,MM,CRT,Mentation,Vomiting,Diarrhea,Inappetence,Lethargic,Lameness,Muscle_pain,Joint_swelling,Reported_weight_loss,Skin_condition,4Dx_tested
0,2 years,7,101,146,137,Pale,UTO,Dull/Depressed,None,None,None,None,None,None,None,Present,Normal,Not Tested
1,7 months,45,102,116,38,Light Pink,>2 sec,Dull/Depressed,Mild,Mild,Moderate,None,None,Mild,None,None,Normal,Not Tested
2,2 years,63,102,161,44,Pink,1-2 sec,QAR,Mild,None,None,Mild,None,None,None,None,Normal,Not Tested
3,1 month,39,101,129,49,Pink,1-2 sec,QAR,Mild,Mild,Mild,Moderate,None,None,None,None,Normal,Not Tested
4,5 months,21,102,165,128,Pale,UTO,QAR,None,None,None,Mild,None,None,None,None,Normal,Not Tested


In [59]:
# Remove the `Not Tested` 4Dx status
noTest_mask = cleaned_df1['4Dx_tested'] != 'Not Tested'
tested_df = cleaned_df1.loc[noTest_mask]

tested_df.shape[0]

16979

In [60]:
tested_df.columns

Index(['Age', 'Weight_lbs', 'Temperature', 'Heart_Rate_bpm', 'Resp_Rate_bpm',
       'MM', 'CRT', 'Mentation', 'Vomiting', 'Diarrhea', 'Inappetence',
       'Lethargic', 'Lameness', 'Muscle_pain', 'Joint_swelling',
       'Reported_weight_loss', 'Skin_condition', '4Dx_tested'],
      dtype='object')

In [61]:
from tables.tests.test_suite import test
# Splitting the age column to separate numbers from words
tested_df[['Age_num', 'age_str']] = tested_df['Age'].apply(lambda x: pd.Series(str(x).split(" ")))
tested_df['Age'] = tested_df['Age_num']
tested_df['Age'] = tested_df['Age'].astype('float')

print(tested_df["age_str"].unique())

# Changing the ages to all be in years
tested_df.loc[tested_df.age_str == 'days', ['Age']] = tested_df['Age'] / 365
tested_df.loc[tested_df.age_str == 'day', ['Age']] = tested_df['Age'] / 365
tested_df.loc[tested_df.age_str == 'months', ['Age']] = tested_df['Age'] / 12
tested_df.loc[tested_df.age_str == 'month', ['Age']] = tested_df['Age'] / 12
tested_df.loc[tested_df.age_str == 'weeks', ['Age']] = tested_df['Age'] / 52
tested_df.loc[tested_df.age_str == 'week', ['Age']] = tested_df['Age'] / 52

tested_df.head()

['year' 'years' 'days' 'months' 'month' 'weeks' 'week' 'day']


,Age,Weight_lbs,Temperature,Heart_Rate_bpm,Resp_Rate_bpm,MM,CRT,Mentation,Vomiting,Diarrhea,Inappetence,Lethargic,Lameness,Muscle_pain,Joint_swelling,Reported_weight_loss,Skin_condition,4Dx_tested,Age_num,age_str
6,1.000000,15,102,183,44,Pink,1-2 sec,QAR,Mild,Mild,None,Mild,None,None,None,None,Normal,Negative,1,year
8,2.000000,72,100,138,21,Pink,>2 sec,BAR,Mild,None,Severe,None,None,None,None,Present,Normal,Negative,2,years
10,0.005479,63,102,160,50,Pink,1-2 sec,QAR,None,None,Mild,None,None,None,Mild,None,Normal,Negative,2,days
11,0.005479,59,102,157,18,Pink,>2 sec,Dull/Depressed,Mild,Mild,None,None,None,None,None,None,Normal,Negative,2,days
19,6.000000,42,104,110,40,Light Pink,1-2 sec,Dull/Depressed,None,Moderate,Mild,None,Present,Moderate,Moderate,Present,Bruising,Positive,6,years


In [62]:
tester = tested_df.drop(['age_str', 'Age_num'], axis=1)
tester.columns

Index(['Age', 'Weight_lbs', 'Temperature', 'Heart_Rate_bpm', 'Resp_Rate_bpm',
       'MM', 'CRT', 'Mentation', 'Vomiting', 'Diarrhea', 'Inappetence',
       'Lethargic', 'Lameness', 'Muscle_pain', 'Joint_swelling',
       'Reported_weight_loss', 'Skin_condition', '4Dx_tested'],
      dtype='object')

In [63]:
tester.head()

,Age,Weight_lbs,Temperature,Heart_Rate_bpm,Resp_Rate_bpm,MM,CRT,Mentation,Vomiting,Diarrhea,Inappetence,Lethargic,Lameness,Muscle_pain,Joint_swelling,Reported_weight_loss,Skin_condition,4Dx_tested
6,1.000000,15,102,183,44,Pink,1-2 sec,QAR,Mild,Mild,None,Mild,None,None,None,None,Normal,Negative
8,2.000000,72,100,138,21,Pink,>2 sec,BAR,Mild,None,Severe,None,None,None,None,Present,Normal,Negative
10,0.005479,63,102,160,50,Pink,1-2 sec,QAR,None,None,Mild,None,None,None,Mild,None,Normal,Negative
11,0.005479,59,102,157,18,Pink,>2 sec,Dull/Depressed,Mild,Mild,None,None,None,None,None,None,Normal,Negative
19,6.000000,42,104,110,40,Light Pink,1-2 sec,Dull/Depressed,None,Moderate,Mild,None,Present,Moderate,Moderate,Present,Bruising,Positive


In [64]:
tester.dtypes

Age                     float64
Weight_lbs                int64
Temperature               int64
Heart_Rate_bpm            int64
Resp_Rate_bpm             int64
MM                       object
CRT                      object
Mentation                object
Vomiting                 object
Diarrhea                 object
Inappetence              object
Lethargic                object
Lameness                 object
Muscle_pain              object
Joint_swelling           object
Reported_weight_loss     object
Skin_condition           object
4Dx_tested               object
dtype: object

### Split data into training and testing groups

In [66]:
# Creating features
X = tester.drop(columns='4Dx_tested')
X = pd.get_dummies(X)

y = tester['4Dx_tested']

X.head()

,Age,Weight_lbs,Temperature,Heart_Rate_bpm,Resp_Rate_bpm,MM_Light Pink,MM_Pale,MM_Pink,CRT_1-2 sec,CRT_<1 sec,...,Muscle_pain_None,Joint_swelling_Mild,Joint_swelling_Moderate,Joint_swelling_None,Reported_weight_loss_None,Reported_weight_loss_Present,Skin_condition_Bruising,Skin_condition_Irritation,Skin_condition_Normal,Skin_condition_Petechiae
6,1.000000,15,102,183,44,0,0,1,1,0,...,1,0,0,1,1,0,0,0,1,0
8,2.000000,72,100,138,21,0,0,1,0,0,...,1,0,0,1,0,1,0,0,1,0
10,0.005479,63,102,160,50,0,0,1,1,0,...,1,1,0,0,1,0,0,0,1,0
11,0.005479,59,102,157,18,0,0,1,0,0,...,1,0,0,1,1,0,0,0,1,0
19,6.000000,42,104,110,40,1,0,0,1,0,...,0,0,1,0,0,1,1,0,0,0


In [67]:
X.describe()

,Age,Weight_lbs,Temperature,Heart_Rate_bpm,Resp_Rate_bpm,MM_Light Pink,MM_Pale,MM_Pink,CRT_1-2 sec,CRT_<1 sec,...,Muscle_pain_None,Joint_swelling_Mild,Joint_swelling_Moderate,Joint_swelling_None,Reported_weight_loss_None,Reported_weight_loss_Present,Skin_condition_Bruising,Skin_condition_Irritation,Skin_condition_Normal,Skin_condition_Petechiae
count,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,...,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.000000,16979.00000,16979.000000,16979.000000
mean,2.677442,52.787738,100.941987,138.631898,65.820779,0.129218,0.139996,0.730785,0.479298,0.211909,...,0.841981,0.026091,0.133812,0.840097,0.789387,0.210613,0.133989,0.02721,0.830968,0.007833
std,3.062319,31.285347,1.586213,29.759678,44.605111,0.335451,0.346994,0.443565,0.499586,0.408673,...,0.364769,0.159411,0.340461,0.366527,0.407756,0.407756,0.340651,0.16270,0.374792,0.088161
min,0.000000,5.000000,99.000000,88.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,0.666667,29.000000,100.000000,113.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000
50%,2.000000,51.000000,101.000000,138.000000,47.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,4.000000,73.000000,102.000000,164.000000,123.000000,0.000000,0.000000,1.000000,1.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.000000
max,20.000000,200.000000,105.000000,190.000000,150.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [68]:
# Check the balance of the target variable
y.value_counts()

Negative    14853
Positive     2126
Name: 4Dx_tested, dtype: int64

In [70]:
# Splitting data into testing and training sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train))
print(Counter(y_test))

Counter({'Negative': 11096, 'Positive': 1638})
Counter({'Negative': 3757, 'Positive': 488})


In [71]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'Negative': 11096, 'Positive': 11096})

In [72]:
from sklearn.linear_model import LogisticRegression

LG_model = LogisticRegression(solver='lbfgs', random_state=1)
LG_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [73]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

y_pred = LG_model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

Confusion_matrix = confusion_matrix(y_test, y_pred)
Confusion_matrix

0.9998669150918286


array([[3756,    1],
       [   0,  488]])

In [74]:
Confusion_matrix_df = pd.DataFrame(Confusion_matrix, index=["Actual Positive", "Actual Negative"], columns=["Predicted Positive", "Predicted Negative"])
Confusion_matrix_df

,Predicted Positive,Predicted Negative
Actual Positive,3756,1
Actual Negative,0,488


In [75]:
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

   Negative       1.00      1.00      1.00      1.00      1.00      1.00      3757
   Positive       1.00      1.00      1.00      1.00      1.00      1.00       488

avg / total       1.00      1.00      1.00      1.00      1.00      1.00      4245

